#### Notebook to prepair raw data  
date: 2023

#### Step 0. Imports


In [1]:
import pandas as pd  
import numpy as np  
from tqdm import tqdm
import os
import random

In [2]:
os.chdir(r"C:\Users\Administrateur\OneDrive\Documents\GitHub\reparatorAI")

#### Step1. Functions  

In [3]:
def first_clean_df(df):
    df_ok = df.copy()
    df_ok['brand']=['-'.join(str(my_brand).upper().strip().split(' ')[0:1]) for my_brand in df_ok.brand]
    df_ok['product_category']=[str(my_val).upper().strip() for my_val in df_ok.product_category]
    my_brand_low_freq = df_ok.brand.value_counts()[df_ok.brand.value_counts() < 5].index.tolist()
    df_ok = df_ok[df_ok.brand.isin(my_brand_low_freq) == False]
    df_ok['repair_barrier_if_end_of_life'] = df_ok['repair_barrier_if_end_of_life'].fillna('Unspecified')
    return df_ok

#### Step2. Datasets  

In [4]:
raw=pd.read_csv('./data/OpenRepairData_v0.3_aggregate_202407.csv', encoding='latin', low_memory=False)

In [5]:
raw.shape

(208491, 14)

In [6]:
raw.columns

Index(['id', 'data_provider', 'country', 'partner_product_category',
       'product_category', 'product_category_id', 'brand',
       'year_of_manufacture', 'product_age', 'repair_status',
       'repair_barrier_if_end_of_life', 'group_identifier', 'event_date',
       'problem'],
      dtype='object')

In [13]:
old=pd.read_csv('./data/OpenRepairData_v0.3_aggregate_202303.csv', index_col=0, encoding='latin', low_memory=False)

In [14]:
old.shape

(103076, 21)

In [15]:
old.columns

Index(['id', 'data_provider', 'country', 'partner_product_category',
       'product_category', 'product_category_id', 'brand',
       'year_of_manufacture', 'product_age', 'repair_status',
       'repair_barrier_if_end_of_life', 'group_identifier', 'event_date',
       'problem', 'product_category_new', 'brand_ok', 'TopCategory',
       'problem_class', 'TopCategory_FR', 'product_category_FR',
       'problem_class_main'],
      dtype='object')

In [16]:
selectObjectList_UK=['POWER TOOL', 'TOY', 'HAIR DRYER', 'DECORATIVE OR SAFETY LIGHTS', 'LAMP',
 'PORTABLE RADIO', 'HANDHELD ENTERTAINMENT DEVICE', 'FOOD PROCESSOR', 'SMALL HOME ELECTRICAL',
 'HAIR & BEAUTY ITEM', 'MISC', 'SEWING MACHINE', 'WATCH/CLOCK', 'HI-FI SEPARATES',
 'DESKTOP COMPUTER', 'BATTERY/CHARGER/ADAPTER', 'SMALL KITCHEN ITEM', 'VACUUM',
 'TV AND GAMING-RELATED ACCESSORIES', 'COFFEE MAKER', 'KETTLE', 'IRON',
 'DIGITAL COMPACT CAMERA', 'PRINTER/SCANNER', 'LAPTOP', 'HI-FI INTEGRATED',
 'PAPER SHREDDER', 'TOASTER', 'FLAT SCREEN', 'MOBILE', 'TABLET', 'DSLR/VIDEO CAMERA',
 'HEADPHONES', 'LARGE HOME ELECTRICAL', 'MUSICAL INSTRUMENT', 'PROJECTOR',
 'PC ACCESSORY', 'AIRCON/DEHUMIDIFIER', 'FAN', 'GAMES CONSOLE']

selectObjectList_FR=['Outil Bricolage', 'Jouet', 'Sèche cheveux', 'Luminaires et guirlandes déco',
 'Lampe', 'Radio portable', 'Appareil de divertissement portable', 'Robot de cuisine', 'Petit électroménager de maison',
 'équipement pour cheveux & beauté', 'Divers', 'Machine à coudre', 'Montre / Réveil', 'Composants HI-Fi', 'Ordinateur de Bureau',
 'Batterie / chargeur / adaptateur', 'Petit électroménager de cuisine', 'Aspirateur', 'accessoire TV et jeux videos', 'Machine à café',
 'Bouilloire', 'Fer à repasser', 'Appareil photo numérique', 'Imprimante / scanner', 'Ordinateur portable', 'Hi-Fi', 'Broyeuse à papier',
 'Grille pains', 'Ecran plat', 'Téléphone portable', 'Tablette', 'Camescope', 'Ecouteurs', 'Gros électroménager',
 'Instrument de musique', 'Vidéo projecteur', 'Accessoire PC', 'Climatiseur / déshumidificateur', 'Ventilateur', 'Console de jeux vidéo']
selectObjectList_FR=[my_str.upper() for my_str in selectObjectList_FR]

In [17]:
topCategory_uk=['BATHROOM', 'ELECTRONICS', 'HOME', 'IMAGE', 'KITCHEN', 'OFFICE', 'OTHER', 'SOUND']
topCategory_fr=['SALLE DE BAIN','ELECTRONIQUE','MAISON','IMAGE','CUISINE','BUREAU','AUTRES','SON']

#### Step3. final treatment and saving  
New columns to create : 'product_category_new', 'brand_ok', 'TopCategory',
       'problem_class', 'TopCategory_FR', 'product_category_FR',
       'problem_class_main'

##### step3.1: create dict objects to map the final dataset   


In [18]:
dict_topCategory=old.groupby(by=['TopCategory','product_category_new']).count().iloc[:,0].index.tolist()
dict_topCategory_all={}
for obj in dict_topCategory:
    dict_topCategory_all[obj[1]]=obj[0]

In [19]:
dict_topCategory_ENtoFR={}
for i in range(len(topCategory_uk)):
    dict_topCategory_ENtoFR[topCategory_uk[i]]=topCategory_fr[i]

In [20]:
dict_objList_ENtoFR={}
for i in range(len(selectObjectList_UK)):
    dict_objList_ENtoFR[selectObjectList_UK[i]]=selectObjectList_FR[i]

##### step3.2: map clean dataset with dict objects

In [21]:
new=first_clean_df(raw)
new["TopCategory"]=new["product_category"].map(dict_topCategory_all)
new["TopCategory_FR"]=new["TopCategory"].map(dict_topCategory_ENtoFR)
new["product_category_FR"]=new["product_category"].map(dict_objList_ENtoFR)

In [22]:
new.head()

,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem,TopCategory,TopCategory_FR,product_category_FR
0,anstiftung_23571,anstiftung,DEU,Unterhaltungselektronik ~ Playstation/Spieleko...,HI-FI SEPARATES,13,SONY,NaN,NaN,End of life,Unspecified,35862,2024-07-29,"Keine Funktion mehr, da CD nicht mehr ausgewor...",SOUND,SON,COMPOSANTS HI-FI
2,anstiftung_23572,anstiftung,DEU,HaushaltsgerÃ¤te ~ Toaster,TOASTER,31,AEG,2014.0,10.0,Fixed,Unspecified,35862,2024-07-29,Temperatur lÃ¤Ãt sich nicht mehr regeln. Mech...,KITCHEN,CUISINE,GRILLE PAINS
3,anstiftung_23596,anstiftung,DEU,HaushaltsgerÃ¤te ~ Wassersprudler,SMALL KITCHEN ITEM,29,SODASTREAM,NaN,NaN,End of life,Unspecified,4571,2024-07-27,Wassersprudler sprudelt nicht mehr. - Druckven...,KITCHEN,CUISINE,PETIT ÉLECTROMÉNAGER DE CUISINE
4,anstiftung_23611,anstiftung,DEU,HaushaltsgerÃ¤te ~ GlÃ¤tteisen,HAIR & BEAUTY ITEM,9,UNKNOWN,NaN,NaN,End of life,Unspecified,4571,2024-07-27,HaarglÃ¤tter funktioniert seit Sturz auf Boden...,BATHROOM,SALLE DE BAIN,ÉQUIPEMENT POUR CHEVEUX & BEAUTÉ
5,anstiftung_23614,anstiftung,DEU,Elektro divers ~ Heckenschere,POWER TOOL,24,UNKNOWN,NaN,NaN,End of life,Unspecified,4571,2024-07-27,Akku-Heckenschere funktioniert nicht mehr. Bit...,HOME,MAISON,OUTIL BRICOLAGE


In [23]:
new.shape

(194495, 17)

##### step3.3: map key words to create clean keywords columns  
1. clean kw dataset - done
2. build proper columns: problem_class and problem_class_main

In [24]:
kw=pd.read_csv('./data/kw_tot_clean.csv', index_col=0)
kw_clean=kw.dropna(subset="CLASS1", axis=0)

In [25]:
word_category_mapping = dict(zip(kw_clean['original'], kw_clean['CLASS1']))

In [26]:
# Initialize a defaultdict to store category counts for each sentence

results=[]
# Iterate over each sentence in datasetB
for i in tqdm(range(new.shape[0])):
    sentence=new['problem'].iloc[i]
    try:
        # Tokenize the sentence into words
        words = sentence.split()
        # Count categories for each word in the sentence
        sentence_category_counts = {}
        for word in words:
            word=word.upper()
            if word in word_category_mapping:
                category = word_category_mapping[word]
                try:
                    sentence_category_counts[category] += 1
                except: 
                    sentence_category_counts[category] = 1
    except:
        sentence_category_counts={}
    results.append(sentence_category_counts)

100%|███████████████████████████████████████████████████████████████████████| 194495/194495 [00:03<00:00, 60252.30it/s]


In [27]:
new['problem_class']=results

In [28]:
def get_keys_with_max_count(dictionary):
    if pd.isnull(dictionary) or not dictionary:
        return 'U' #unknown
    
    # Find the key(s) with the maximum count
    max_keys = max(dictionary, key=dictionary.get, default=None)
    
    # If max_keys is not None, return it; otherwise, return an empty list
    return [max_keys][0] if max_keys is not None else 'U'

In [29]:
new['problem_class_main']=new['problem_class'].apply(lambda x: get_keys_with_max_count(x))

##### Step3.4: save the dataset  

In [30]:
new.to_csv(r'./data/OpenRepairData_v0.3_aggregate_202407.csv')

#### Step4. clean the brands  


In [31]:
df=pd.read_csv(r'./data/OpenRepairData_v0.3_aggregate_202407.csv', index_col=0)

In [32]:
df.head(2)

,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem,TopCategory,TopCategory_FR,product_category_FR,problem_class,problem_class_main
0,anstiftung_23571,anstiftung,DEU,Unterhaltungselektronik ~ Playstation/Spieleko...,HI-FI SEPARATES,13,SONY,NaN,NaN,End of life,Unspecified,35862,2024-07-29,"Keine Funktion mehr, da CD nicht mehr ausgewor...",SOUND,SON,COMPOSANTS HI-FI,"{'G': 1, 'E': 1, 'M': 1}",G
2,anstiftung_23572,anstiftung,DEU,HaushaltsgerÃ¤te ~ Toaster,TOASTER,31,AEG,2014.0,10.0,Fixed,Unspecified,35862,2024-07-29,Temperatur lÃ¤Ãt sich nicht mehr regeln. Mech...,KITCHEN,CUISINE,GRILLE PAINS,{'M': 1},M


In [33]:
df.brand.value_counts()

brand
UNKNOWN     80317
PHILIPS      9932
BOSCH        4064
SONY         3723
HP           3029
            ...  
TONDEO          5
APEMAN          5
ISCOOTER        5
EXOST           5
NIET            5
Name: count, Length: 1792, dtype: int64

In [34]:
list_brand_unique=df.brand.unique()

In [35]:
from difflib import SequenceMatcher
from collections import Counter

def get_similarity_score(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

def find_similar_brands(brand_list):
    similar_brands_dict = {}

    for i in tqdm(range(len(brand_list))):
        current_brand = brand_list[i]
        similar_brands = {}

        for j in range(len(brand_list)):
            if i != j:
                other_brand = brand_list[j]
                similarity_score = get_similarity_score(current_brand, other_brand)

                # You can adjust the threshold as needed
                if similarity_score >= 0.9:
                    similar_brands[other_brand] = similarity_score

        if similar_brands:
            similar_brands_dict[current_brand] = similar_brands

    return similar_brands_dict

In [36]:
def clean_brand_list(brands):
    brand_frequencies = Counter(brands)
    unique_brands = list(set(brands))
    similar_brands_dict = find_similar_brands(unique_brands)

    for current_brand, similar_brands in similar_brands_dict.items():
        if similar_brands:
            most_frequent_brand = max([current_brand] + list(similar_brands), key=lambda x: brand_frequencies[x])

            for brand in similar_brands:
                if brand_frequencies[brand] < brand_frequencies[most_frequent_brand]:
                    # Replace low-frequency brands with the most frequent one
                    brands = [most_frequent_brand if x == brand else x for x in brands]

    return brands

In [37]:
new_brands=clean_brand_list(df.brand.tolist())

100%|██████████████████████████████████████████████████████████████████████████████| 1792/1792 [00:51<00:00, 34.56it/s]


In [38]:
df["brand"]=new_brands
df.to_csv(r'./data/OpenRepairData_v0.3_aggregate_202407.csv')

In [1]:
selectObjectList_FR=['Outil Bricolage', 'Jouet', 'Sèche cheveux', 'Luminaires et guirlandes déco',
 'Lampe', 'Radio portable', 'Appareil de divertissement portable', 'Robot de cuisine', 'Petit électroménager de maison',
 'équipement pour cheveux & beauté', 'Divers', 'Machine à coudre', 'Montre / Réveil', 'Composants HI-Fi', 'Ordinateur de Bureau',
 'Batterie / chargeur / adaptateur', 'Petit électroménager de cuisine', 'Aspirateur', 'accessoire TV et jeux videos', 'Machine à café',
 'Bouilloire', 'Fer à repasser', 'Appareil photo numérique', 'Imprimante / scanner', 'Ordinateur portable', 'Hi-Fi', 'Broyeuse à papier',
 'Grille pains', 'Ecran plat', 'Téléphone portable', 'Tablette', 'Camescope', 'Ecouteurs', 'Gros électroménager',
 'Instrument de musique', 'Vidéo projecteur', 'Accessoire PC', 'Climatiseur / déshumidificateur', 'Ventilateur', 'Console de jeux vidéo']
selectObjectList_FR=[my_str.upper() for my_str in selectObjectList_FR]

In [3]:
print(selectObjectList_FR)

['OUTIL BRICOLAGE', 'JOUET', 'SÈCHE CHEVEUX', 'LUMINAIRES ET GUIRLANDES DÉCO', 'LAMPE', 'RADIO PORTABLE', 'APPAREIL DE DIVERTISSEMENT PORTABLE', 'ROBOT DE CUISINE', 'PETIT ÉLECTROMÉNAGER DE MAISON', 'ÉQUIPEMENT POUR CHEVEUX & BEAUTÉ', 'DIVERS', 'MACHINE À COUDRE', 'MONTRE / RÉVEIL', 'COMPOSANTS HI-FI', 'ORDINATEUR DE BUREAU', 'BATTERIE / CHARGEUR / ADAPTATEUR', 'PETIT ÉLECTROMÉNAGER DE CUISINE', 'ASPIRATEUR', 'ACCESSOIRE TV ET JEUX VIDEOS', 'MACHINE À CAFÉ', 'BOUILLOIRE', 'FER À REPASSER', 'APPAREIL PHOTO NUMÉRIQUE', 'IMPRIMANTE / SCANNER', 'ORDINATEUR PORTABLE', 'HI-FI', 'BROYEUSE À PAPIER', 'GRILLE PAINS', 'ECRAN PLAT', 'TÉLÉPHONE PORTABLE', 'TABLETTE', 'CAMESCOPE', 'ECOUTEURS', 'GROS ÉLECTROMÉNAGER', 'INSTRUMENT DE MUSIQUE', 'VIDÉO PROJECTEUR', 'ACCESSOIRE PC', 'CLIMATISEUR / DÉSHUMIDIFICATEUR', 'VENTILATEUR', 'CONSOLE DE JEUX VIDÉO']


In [47]:
my_quotes_data=pd.read_csv('data/quotes.csv', sep=';', encoding='latin')

In [48]:
my_quotes_data.shape

(165, 4)

In [29]:
def get_random_row(dataset):
    """
    Get a random row from the dataset.

    Args:
    - dataset: A list of lists or a pandas DataFrame representing the dataset.

    Returns:
    - A random row from the dataset.
    """
    if isinstance(dataset, list):
        return random.choice(dataset)
    elif isinstance(dataset, pd.DataFrame):
        return dataset.sample(n=1)
    else:
        raise TypeError("Dataset should be a list of lists or a pandas DataFrame.")

In [45]:
def get_quote(the_data, the_product, lang_var):
    the_data_ok=the_data[the_data.lang==lang_var]
    the_data_ok=the_data_ok[the_data_ok.Equipment==the_product]
    num=the_data_ok.shape[0]
    the_quote=get_random_row(the_data_ok)
    author=the_quote.Author.values[0]
    quote=the_quote.quote.values[0]
    return author, quote

In [46]:
get_quote(my_quotes_data, "LAMP", "UK")

('J.K. Rowling',
 "A lamp is like a wizard's wand, illuminating even the darkest corners of a room.")

In [20]:
import random

In [28]:
random.sample(range(,1)

[4]